In [2]:
import argparse
import cv2
import matplotlib.pyplot as plt
import numpy as np
import torchvision as torchvision


images = np.ones((1,183, 275, 3))
for i in range(1,6):
    path = 'BlurredImages/blurred0' + str(i) +'.jpg'
    print(path)
    image = cv2.imread(path)[:183, :275, :3]
    
    #gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = image.reshape((1, 183, 275, 3))
    
    images = np.concatenate((images, image), axis = (0))
    
    
images = images.astype(np.uint8)
batch = images.copy()                         # trasformazione di batch di immagini normali(batch) in batch di immagini grayscale
images = np.ones((1,183, 275))                # images perchè cvtColor non accetta batch
for img in batch:
    print(img.shape)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = gray.reshape((1, 183, 275))
    
    images = np.concatenate((images, gray), axis = (0))
    
images = images[2:]


BlurredImages/blurred01.jpg
BlurredImages/blurred02.jpg
BlurredImages/blurred03.jpg
BlurredImages/blurred04.jpg
BlurredImages/blurred05.jpg
(183, 275, 3)
(183, 275, 3)
(183, 275, 3)
(183, 275, 3)
(183, 275, 3)
(183, 275, 3)


In [9]:
images = torchvision.io.read_video('Kobe.mp4', end_pts=100, pts_unit='sec') 
images = images[0].numpy()
#images = images.astype(np.uint8)

In [11]:
images = cv2.convertScaleAbs(images) 
for i in range(images.shape[0]):
    #print(images[i].shape)
    cv2.imshow("Image", images[i])
    cv2.waitKey(0)
    cv2.destroyAllWindows()

KeyboardInterrupt: 

In [12]:
images.shape

(2998, 720, 1280, 3)

In [ ]:
batch = images.copy()                         # trasformazione di batch di immagini normali(batch) in batch di immagini grayscale
images = np.ones((1,batch.shape[1], batch.shape[2]))                # images perchè cvtColor non accetta batch
for img in batch:
    #print(img.shape)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = gray.reshape((1, batch.shape[1],batch.shape[2]))
    
    images = np.concatenate((images, gray), axis = (0))
    
images = images[1:]

In [23]:
images.shape

(150, 720, 1280)

In [24]:
i = 0
cap = cv2.VideoCapture('Kobe.mp4')

images = np.ones((1,720, 1280)) 
while(cap.isOpened()):
    ret, frame = cap.read()

    #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = gray.reshape((1, gray.shape[0],gray.shape[1]))
    
    
    images = np.concatenate((images, gray), axis = (0))
    
    i += 1
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



In [25]:
frame.shape


(720, 1280, 3)

In [26]:
images.shape

(43, 720, 1280)

In [27]:
images = cv2.convertScaleAbs(images) 
for i in range(images.shape[0]):
    cv2.imshow("Image", images[i])
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cv2.destroyAllWindows()

In [28]:
images = images[:,np.newaxis]

images.shape



(43, 1, 720, 1280)

In [36]:
import torch
def convolution(input, kernel):
    
    n, iC, H, W = input.shape
    kH = kernel.shape[2]
    kW = kernel.shape[3]
    oC = kernel.shape[0]
    
    out = torch.zeros((n, oC, (H - kH + 1), (W - kW + 1))).cuda()
    for r in range(0, H - kH + 1):
        for c in range(0, W - kW + 1):
            out[:,:, r, c] = (input[:,:,r:r+kH,c:c+kW].reshape((n, 1, iC, kH, kW)) * kernel.reshape((1, oC, iC, kH, kW))).sum((2, 3, 4))
            #print(res.shape, (r, c))
    return out

In [37]:
i = int(input('which kernel : '))
if i == 1:
    kernel = np.array([[0, -1, 0],
                  [-1, 4, -1], 
                  [0, -1, 0]], dtype=np.float32)
else:
    kernel = np.array([[-1, -1, -1],
                  [-1, 8, -1], 
                  [-1, -1, -1]], dtype=np.float32)


which kernel : 1


In [38]:
kernel

array([[ 0., -1.,  0.],
       [-1.,  4., -1.],
       [ 0., -1.,  0.]], dtype=float32)

In [39]:
k  = kernel + np.zeros((3, 3))
k = k[np.newaxis,np.newaxis]

In [40]:
k.shape

(1, 1, 3, 3)

In [41]:
images.shape

(43, 1, 720, 1280)

In [56]:
o = convolution(torch.from_numpy(images).cuda(), torch.from_numpy(k).cuda())
o.shape

torch.Size([43, 1, 718, 1278])

In [45]:
o = o.cpu().numpy()
o = np.squeeze(o)
o = cv2.convertScaleAbs(o) 
for i in range(o.shape[0]):
    cv2.imshow("Image", o[i])
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [46]:
vars = np.var(o, axis=(1, 2))

In [47]:
mask = vars >= 140

In [48]:
images[mask].shape

(42, 1, 720, 1280)

In [49]:
import torch
torch.cuda.is_available()

True

In [50]:
torch.cuda.current_device()

0

In [51]:
torch.cuda.device_count()

1

In [52]:
torch.cuda.get_device_name(device=0)

'NVIDIA GeForce GTX 1050 Ti'

In [53]:
torch.cuda.current_device()

0

In [54]:
var = torch.FloatTensor([1.0, 2.0, 3.0]).cuda()

In [55]:
var.device

device(type='cuda', index=0)